### ACTUALIZACIÓN: Acerca de esta lección

*El objetivo de esta lección era practicar un despliegue en la nube utilizando un servicio reconocido llamado Heroku que desde hace años proporcionaba una versión de prueba sin coste. Lamentablemente la empresa ha decidido dar de baja los planes gratuitos a partir del 28 de Noviembre de 2022:*

[![](docs/img21.png)](https://www.heroku.com/pricing)

*Mi intención al crear los apuntes era compartir con vosotros una demostración práctica sin ningún coste, pero debido a estos cambios el precio para realizar la práctica pasará a ser mínimo de `$7` por el servicio más `$10` de la base de datos (mensuales). Si en algún momento Heroku recupera las pruebas gratuitas grabaré esta lección en vídeo, todo es completamente funcional a fecha de Agosto de 2022.*

# Despliegue en internet con Heroku

Tener Django en nuestra máquina está bien pero sería mucho mejor tenerlo en Internet para que cualquiera pueda acceder a nuestro blog.

En esta lección aprenderemos a desplegar un proyecto en Heroku, una plataforma de cloud computing que permite manejar servicios web en la nube. Es un proceso largo que requiere un esfuerzo adicional por vuestra parte, así que muchos ánimos.

## Git y Heroku CLI

Para realizar un despliegue debemos tener el proyecto configurado en un gestor de versiones como [Git](https://git-scm.com/downloads).

Git suele venir instalado en **MAC** y **Linux**, en **Windows** es necesario descargarlo de la web oficial [https://git-scm.com/downloads](https://git-scm.com/downloads) y seguir los pasos del instalador, añadiendo el programa al PATH del sistema si lo pide en algún momento.

Una vez tengáis `Git` instalado y accesible en la terminal podemos continuar:

![](docs/img06.png)

El siguiente paso es crear una cuenta en https://www.heroku.com/ y conseguir acceso a vuestro *dashboard* en https://dashboard.heroku.com/.

Para desplegar proyectos necesitamos instalar el intérprete `Heroku CLI` que es desde donde se ejecutan las instrucciones. La instalación de este programa depende del sistema operativo, deberéis seguir las instrucciones de la página oficial [https://devcenter.heroku.com/articles/heroku-cli](https://devcenter.heroku.com/articles/heroku-cli).

En **Windows** es descargar el instalador y seguir los pasos:

* 64 bits: https://cli-assets.heroku.com/heroku-x64.exe
* 32 bits: https://cli-assets.heroku.com/heroku-x86.exe

En **MAC** se puede instalar mediante [Homebrew](https://brew.sh/index_es):

```bash
$ brew tap heroku/brew && brew install heroku
```

Y en **Linux** utilizando el script oficial:

```bash
$ curl https://cli-assets.heroku.com/install.sh | sh
```

En mi caso ya lo tengo todo listo, vosotros deberéis hacerlo por vuestra cuenta, si tenéis algún problema decídmelo y os ayudo en lo que pueda.

Cuando tengáis `Heroku CLI` instalado y accesible desde la terminal podemos continuar:

![](docs/img05.png)

## Preparando el proyecto

Un proyecto de Django requiere algunos ajustes para poder ser desplegado en `Heroku`, es lo que se conoce como la configuración para producción.

Como quiero que todos partamos del mismo proyecto he comprimido mi proyecto en un fichero `zip`, podéis descargarlo en los recursos de esta lección, contiene un fichero `requirements.txt` con las dependencias.

En él vamos a crear un nuevo entorno virtual instalando las dependencias:

```bash
$ cd tutorial-django-blog/
$ pipenv install -r requirements.txt
```

Si se instala correctamente podéis poner en marcha el proyecto:

```bash
$ pipenv run python manage.py runserver
```

Si todo está *ok* podemos pasar a los ajustes de producción.

## Ajustes de producción

Según el [tutorial oficial de despliegue de Django en Heroku](https://devcenter.heroku.com/articles/django-app-configuration), el proyecto necesita un fichero `Procfile` donde se define el tipo de servicio a desplegar.

Heroku permite 4 tipos de servicio:

* `web`: Recibe tráfico HTTP.
* `worker`: Realiza un trabajo en segundo plano.
* `clock`: Ejecuta un trabajo programado.
* `release`: Ejecuta una tarea antes de la implementación.

Django es un servicio `web`, así que la configuración del nuevo fichero `Procfile` contendrá lo siguiente:

`Procfile`

```
web: python manage.py runserver 0.0.0.0:\$PORT
```

Esto significa que se inicie el servicio en la dirección y puerto por defecto. En produccion el puerto lo asigna  dinámicamente Heroku, de ahí la variable `$PORT`.

Si probamos la configuración, al estar trabajando en un entorno virtual no funcionará:

```bash
$ heroku local web
```

```
ModuleNotFoundError: No module named 'django'
```

Para solucionar esto vamos a trabajar con un servidor WSGI intermediario que hará de puerta de enlace entre las peticiones y Django funcionando en el entorno, se llama `gunicorn`, vamos a instalarlo:

```bash
$ pipenv install gunicorn
```

Podemos ejecutar el servicio de Django mediante `gunicorn` con el comando:

```bash
pipenv run gunicorn tutorial.wsgi
```

Esto irá a buscar la configuración en el directorio `tutorial/wsgi.py`. El servicio funcionará pero los ficheros estáticos no, luego lo arreglamos.

Vamos a añadir una configuración para la fase de desarrollo a la que podemos llamar `dev`, aquí debemos especificar el puerto manualmente ya que por defecto se utiliza el 5000 y puede estar en uso:

`Procfile`

```
web: python manage.py runserver 0.0.0.0:\$PORT
dev: pipenv run gunicorn tutorial.wsgi -b 127.0.0.1:8000
```

Esto debería funcionar al ejecutar:

```bash
$ heroku local dev
```

Como vimos antes los ficheros estáticos no se están sirviendo, las imágenes y CSS no se cargarán:

![](docs/img07.png)

Esto sucede porque `gunicorn` no sabe donde ir a buscar los ficheros, pues estos se encuentran cargados en la memoria de cada app de Django. 

Necesitamos recopilarlos en un directorio común, para ello vamos a crear una carpeta en la raíz llamada `staticfiles` cuya ruta configuraremos en la variable `STATIC_ROOT` en `settings.py`:

```python
STATIC_ROOT = BASE_DIR / 'staticfiles'
```

Para recopilar los ficheros estáticos ejecutaremos el comando `collectstatic`:

```
$ pipenv run python manage.py collectstatic
```

```
132 static files copied to '/Users/hektor/Proyectos/tutorial-django-blog/staticfiles'.
```

![](docs/img08.png)

Si ponemos en marcha el entorno `dev` debería funcionar:

```bash
$ heroku local dev
```

![](docs/img09.png)

Según la [documentación de Heroku](https://devcenter.heroku.com/articles/django-assets) el comando `collectstatic` se ejecuta automáticamente después de realizar el despliegue en producción, así que por lo menos algo nos vamos a ahorrar.

Pero lástimosamente si cambiamos `DEBUG = False` y permitimos cualquier dominio como origen `ALLOWED_HOSTS = ["*"]` en el `settings.py`, como Django no sirve ficheros estáticos en producción volverá a dejar de funcionar (en caso que os funcione es debido al caché, cerrar el navegador y abridlo de nuevo):

![](docs/img07.png)

Por suerte existe un paquete llamado [WhiteNoise](http://whitenoise.evans.io/en/stable/) que permite a los servicios web servir ficheros estáticos en producción, podemos instalarlo:

```bash
$ pipenv install whitenoise
```

Este módulo contiene un middleware para Django que lo manejará todo por nosotros, simplemente debemos activarlo justo debajo del middleware de seguridad:

```python
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'whitenoise.middleware.WhiteNoiseMiddleware', # <---
]
```

Abajo del todo en el `settings.py` añadimos esta línea para activar la compresión:

```python
STATICFILES_STORAGE = "whitenoise.storage.CompressedStaticFilesStorage"
```

Con esto debería servir los ficheros estáticos sin el modo `DEBUG`:

```bash
$ heroku local dev
```

![](docs/img09.png)

## Configuración remota

Nuestro entorno de pruebas local está listo, ahora debemos adaptar el `Procfile` para que Heroku ponga en marcha el proceso `web` mediante `gunicorn`, él mismo se encargará de instalar y configurar el entorno virtual al encontrar el `Pipfile` en el directorio raíz:

`tutorial/Procfile`

```
web: gunicorn tutorial.wsgi --log-file -
test: pipenv run gunicorn tutorial.wsgi -b 127.0.0.1:8000
```

En este punto regeneramos el fichero `Pipenv.lock`:

```bash
$ pipenv lock
```

Y en principio ya estamos listos para desplegar el proyecto.

## Desplegando la aplicación

Antes de continuar os aviso de que vamos a utilizar comandos de `Git`, el sistema de control de versiones. Esta aplicación es de obligado aprendizaje para cualquier programador hoy en día. Solo utilizaremos algunas instrucciones básicas pero os animo encarecidamente a que os toméis una tarde libre para aprender más sobre esta tecnología.

Vamos a identificarnos en **Heroku CLI**:

```bash
$ heroku login
```

Esto abrirá el navegador y nos identificará con la sesión del usuario que tenemos registrado:

```
Logging in... done
Logged in as xxxyyyzzz@gmail.com
```

Ahora vamos a inicializar un repositorio `Git` en el directorio raíz mediante la instrucción:

```bash
$ git init
```

Cambiaremos el nombre de la rama `master` a `main`:

```bash
$ git branch -m main
```

Ahora el directorio del proyecto es un repositorio con versiones, podemos añadir el código inicial con un mensaje de confirmación:

```bash
$ git add . --all
$ git commit -m "Despliegue preparado"
```

En este punto toca trasladar el código local a la nube, para ello necesitamos crear una aplicación en Heroku.

Podemos crear una con manual o automáticamente, tened en cuenta que si elegís un nombre a mano puede estar en uso:

```bash
$ heroku create                # nombre automatico
$ heroku create -a nombre-app  # nombre manual
```

Se creará la aplicación y se añadirá como repositorio remoto:

```
Creating app... done, ⬢ rocky-tundra-25590
https://rocky-tundra-25590.herokuapp.com/ | https://git.heroku.com/rocky-tundra-25590.git
```

Si accedemos al `dashboard` de la página veremos la aplicación:

![](docs/img10.png)

Hacemos clic en el proyecto y en el apartado **Settings**, establecemos el *Buildpack* en `heroku/python` y guardamos:

![](docs/img11.png)

Esto también se puede hacer desde la terminal escribiendo el comando:

```bash
$ heroku buildpacks:set heroku/python
```

Ahora vamos a enlazar el repositorio local con el remoto:

```bash
$ heroku git:remote -a rocky-tundra-25590
```

```
set git remote heroku to https://git.heroku.com/rocky-tundra-25590.git
```

Ha llegado el momento de la verdad, vamos publicar la rama principal en el repositorio remoto:

```bash
$ git push heroku main
```

Heroku buscará todas las dependencias del `Pipfile`, las instalará y si todo funciona correctamente ya podremos acceder a nuestra página web en la URL del servicio https://rocky-tundra-25590.herokuapp.com/:

![](docs/img12.png)

Tened en cuenta que esta aplicación es de prueba, tiene recursos y tiempo de uso limitados, si queréis algo más sofisticado tendréis que pagar y ya os adelanto que Heroku no es precisamente barato. La alternativa es configurar vuestro propio servidor web, aunque eso es harina de otro costal.

## PostgreSQL en Heroku

La página web es completamente funcional, podemos incluso añadir entradas al administrador:

![](docs/img13.png)

Pero hay un pequeño problema y es que [el sistema de ficheros de Heroku es efímero](https://devcenter.heroku.com/articles/dynos#ephemeral-filesystem), eso significa que todos los cambios se deshacen al reiniciar la aplicación y una base de datos SQLite nunca será persistente por el hecho de funcionar en un fichero.

Para ilustrar este inconveniente supongamos que realizamos una modificación del código, como un comentario abajo del `settings.py`:

```python
# Comentario en el settings que no afecta en nada
```

Añadimos el cambio al repositorio:

```bash
$ git add tutorial/settings.py
$ git commit -m "Actualizacion de prueba"
```

```
[main e4f1b1f] Actualizacion de prueba
 1 file changed, 2 insertions(+)
```

Si publicamos la nueva versión del proyecto con este insignificante cambio, éste se reiniciará y...

```bash
$ git push heroku main
```

![](docs/img14.png)

**¡BOOM!** Se habrá borrado la cuarta entrada y todos los cambios.

La solución es utilizar una base de datos en un servicio aislado y por suerte para nosotros como los de Heroku son buena gente nos regalan una para pruebas. Funciona sobre `PostgreSQL`, se llama `Hobby Dev` y tiene unos flamantes 10MB de tamaño. Ya sabéis, si queréis más hay que pagar. 

Esta base de datos se encuentra creada por defecto, podemos observar su configuración en el *dashboard*:

![](docs/img15.png)

Para hacer uso de ella en producción utilizaremos dos paquetes que nos facilitarán mucho la vida `python_decouple` y `dj_database_url`:

```bash
$ pipenv install python_decouple dj_database_url
``` 

Ahora podemos recuperar automáticamente la configuración de la base de datos utilizando como origen la cadena de configuración en las variables de entorno de la aplicación de Heroku:

![](docs/img16.png)

En la parte superior de `settings.py` importaremos ambos módulos:

```python
import dj_database_url
from decouple import config
```

Abajo del todo sobreescribiremos, solo para producción y de una forma poco elegante todo hay que decirlo, la configuración de la base de datos **PostgreSQL**:

```python
if not DEBUG:
    DATABASES = {
        'default': dj_database_url.config(
            default=config('DATABASE_URL')
        )
    }
```

Fijaros que concuerda la variable de entorno `DATABASE_URL` con la que se establece en la conifguración.

Publicamos los cambios:

```bash
$ git add . --all
$ git commit -m "Base de datos postgresql"
$ git push heroku main
```

Veremos que fallará al hacer el despliegue:

```
No module named 'psycopg2'
```

Para desplegar Django con PostgreSQL necesitamos dos cosas, primero instalar esta base de datos en nuestra máquina y luego el conector de Python llamado `psycopg2`, así que no hay más remedio que instalarla.

Si tenéis **Windows** tenéis que ir a la web https://postgresapp.com/downloads.html, descargar el instalador para vuestro sistema y seguir las instrucciones. 

En **Mac** puedo utilizar `brew`:

```bash
$ brew install postgresql
```

En **Linux** se puede utilizar `apt-get` o el gestor de turno:

```bash
$ sudo apt-get install postgresql
```

Una vez instalado, si el servicio está en marcha deberíamos tener acceso a él desde una terminal:

![](docs/img18.png)

En este punto podemos continuar con la configuración de PostgreSQL instalando `pyscopg2` en el entorno virtual:

```bash
$ pipenv install psycopg2-binary
``` 

Publicamos los cambios:

```bash
$ git add . --all
$ git commit -m "Paquete psycopg2"
$ git push heroku main
```

Si accedemos a la página web no funcionará, dará un error 500:

![](docs/img19.png)

Tranquilidad, el problema es que tenemos que realizar la migración inicial. 

Podemos ejecutar comandos de forma remota en el servicio de Heroku muy fácilmente:

```bash
$ heroku run python manage.py migrate
```

Listo, aunque sin usuario ni entradas... 

![](docs/img20.png)

Podemos crear un primer superusuario remotamente y empezar a añadir contenido persistente que no se borrará al actualizar el repositorio:

```bash
$ heroku run python manage.py createsuperuser
```

Esto es muy cómodo, podemos incluso utilizar la `shell` de forma remota (aunque con un poco de lag):

```bash
$ heroku run python manage.py shell
```

Si tenéis algun problema podéis revisar los últimos logs del servicio con el comando:

```bash
$ heroku logs -n 10
```

Para más información sobre la gestión de los servicios de Heroku os dejo la [documentación oficial](https://devcenter.heroku.com/articles/how-heroku-works), es todo un mundo.

En cualquier caso os felicito, ya tenemos el proyecto publicado en la nube, una tarea digna de un administrador profesional. 